In [1]:
pip install jupyter-dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 44.4 MB/s eta 0:00:00


In [2]:
# Importing the necessary libraries
import dash
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [23]:
# Defining the app
app = Dash(__name__)
covid_data = pd.read_csv('/content/sample_data/coviddata.csv')

# Filtering out the unique states
states = covid_data['State'].unique()
print(covid_data)

# Defining the dashbiard layout (all the required dropdowns, radio buttons (selections) required for task 1 and task 2
# and the plots)
app.layout = html.Div([
    html.H1("COVID-19 Data Analysis Dashboard"),

    # Date selection dropdown
    html.Label("Select Date:"),
    dcc.DatePickerSingle(
        id='date-picker',
        min_date_allowed =covid_data['date'].min(),
        max_date_allowed =covid_data['date'].max(),
        initial_visible_month=covid_data['date'].max(),
        date=covid_data['date'].max()
    ),

    # Mode seletion (log or linear)
    html.Label("Select Scale:"),
    dcc.RadioItems(
        id='scale-selector',
        options=[
            {'label': 'Linear', 'value': 'linear'},
            {'label': 'Log', 'value': 'log'}
        ],
        value='linear',
        labelStyle={'display': 'inline-block'}
    ),

    # Choose analysis (cases or deaths)
    html.Label("Select Analysis:"),
    dcc.Checklist(
        id='analysis-selector',
        options=[
            {'label': 'Cases', 'value': 'cases'},
            {'label': 'Deaths', 'value': 'deaths'}
        ],
        value=['cases', 'deaths'],
        labelStyle={'display': 'inline-block'}
    ),

    # Regression selection (linear or non-linear)
    html.Label("Select Regression Mode:"),
    dcc.RadioItems(
        id='regression-selector',
        options=[
            {'label': 'Linear', 'value': 'linear'},
            {'label': 'Polynomial', 'value': 'polynomial'}
        ],
        value='linear',
        labelStyle={'display': 'inline-block'}
    ),

    dcc.Graph(id='main-graph'),

    # Selection of states (multiple selection)
    html.Label("Select State(s):"),
    dcc.Dropdown(
        id='state-selector',
        options=[{'label': state, 'value': state} for state in states],
        value=[states[0]],  # Default value for the dropdown
        multi=True
    ),

    html.Label("Select Data Type:"),
    dcc.RadioItems(
        id='data-type-selector',
        options=[
            {'label': 'Cases', 'value': 'cases'},
            {'label': 'Deaths', 'value': 'deaths'}
        ],
        value='cases',
        labelStyle={'display': 'inline-block'}
    ),

    html.Div([
        dcc.Graph(id='trend-graph')
    ])
])


@app.callback(
    Output('main-graph', 'figure'),
    Input('date-picker', 'date'),
    Input('scale-selector', 'value'),
    Input('analysis-selector', 'value'),
    Input('regression-selector', 'value')
)

# Defining the funtion to accomplish the task 1 (building main graph). Passing the date, mode, regression type as parameters
def update_main_graph(selected_date, selected_scale, selected_analysis, regression_mode):
    filtered = covid_data[covid_data['date'] == selected_date]

    fig = px.scatter()
    for analysis in selected_analysis:
        if analysis == 'cases':
            y = filtered['cases']
        elif analysis == 'deaths':
            y = filtered['deaths']

        fig.add_scatter(x=filtered['cases'], y=y, mode='markers', name=analysis.capitalize())

        # Curve fit for Linear Regression
        if regression_mode == 'linear':
            X = filtered['cases'].values.reshape(-1, 1)
            model = LinearRegression()
            model.fit(X, y)
            y_pred = model.predict(X)

        #Curve fit for Polynomial Regression
        elif regression_mode == 'polynomial':
            X = filtered['cases'].values.reshape(-1, 1)
            poly = PolynomialFeatures(degree=3)  # Considering degree 3 as we have used the same in previous stages.
            X_poly = poly.fit_transform(X)
            model = LinearRegression()
            model.fit(X_poly, y)
            y_pred = model.predict(X_poly)

        fig.add_scatter(x=filtered['cases'], y=y_pred, mode='lines', name=f'{analysis.capitalize()} - {regression_mode.capitalize()} Trend')

    fig.update_layout(title='COVID-19 Analysis', xaxis_title='Cases', yaxis_title='Deaths' if 'deaths' in selected_analysis else 'Cases', showlegend=True, legend_title='Analysis')

    if selected_scale == 'log':
        fig.update_yaxes(type='log')

    return fig

@app.callback(
    Output('trend-graph', 'figure'),
    Input('state-selector', 'value'),
    Input('data-type-selector', 'value')
)

# Defining function to plot the trend line using moving average
def update_trend_graph(selected_states, data_type):
    filtered = covid_data[covid_data['State'].isin(selected_states)]
    filtered[f'{data_type}_ma'] = filtered.groupby('State')[data_type].transform(lambda x: x.rolling(7).mean())

    fig = px.line(filtered, x='date', y=f'{data_type}_ma', color='State', title=f'7-Day Moving Average Trend ({data_type.capitalize()})')

    return fig

if __name__ == '__main__':
    app.run_server(debug=True,port=8051)


        Unnamed: 0  countyFIPS         County Name State  StateFIPS  \
0                0        1001     Autauga County     AL          1   
1                1        1003     Baldwin County     AL          1   
2                2        1005     Barbour County     AL          1   
3                3        1007        Bibb County     AL          1   
4                4        1009      Blount County     AL          1   
...            ...         ...                 ...   ...        ...   
681809      681809       56037  Sweetwater County     WY         56   
681810      681810       56039       Teton County     WY         56   
681811      681811       56041       Uinta County     WY         56   
681812      681812       56043    Washakie County     WY         56   
681813      681813       56045      Weston County     WY         56   

              date  cases  deaths  
0       2020-06-01    233       5  
1       2020-06-01    292       9  
2       2020-06-01    172       1  
3  

<IPython.core.display.Javascript object>